<a href="https://colab.research.google.com/github/AnnPike/final-project-ML/blob/master/extract_CI_ResNet12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
save_to = "/content/gdrive/My Drive/nets_to_compute_smoothness/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
import numpy as np
import time
from torchvision import datasets
import torchvision.transforms as transforms
batch_size = 250
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (1,1,1))])

train_data = datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
print(len(train_data))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=250)

num_train = len(train_data)
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

cuda:0


Extracting data/cifar-10-python.tar.gz to data
50000


In [ ]:
#to extract convolved images
import pickle
def get_activation(name):
    def hook(model, input, output):
      if name in dict_X:
        X = dict_X[name]
        dict_X[name] = torch.cat([X,output.reshape(batch_size,-1).to(device)],axis=0)
      else:
        dict_X[name] = output.reshape(batch_size,-1).to(device)
    return hook

In [ ]:
# model_name = 'ResNet12' # for final model
# model_name = 'ResNet12_5' #for epoch 5
model_name = 'ResNet12_overfitting_100'#for overfited model
class ResNet12(nn.Module):
  def __init__(self):
    super(ResNet12,self).__init__()
    self.conv1 = nn.Conv2d(3, 4, 4, padding=1,stride=2, bias=False)
    self.conv2 = nn.Conv2d(4, 4, 3, padding=1, bias=False)
    self.conv3 = nn.Conv2d(4, 4, 3, padding=1, bias=False)
    self.conv4 = nn.Conv2d(4, 8, 3, padding=1, bias=False)
    self.conv5 = nn.Conv2d(8, 8, 3, padding=1, bias=False)
    self.conv6 = nn.Conv2d(8, 8, 3, padding=1, bias=False)
    self.conv7 = nn.Conv2d(8, 8, 3, padding=1, bias=False)
    self.conv8 = nn.Conv2d(8, 16, 3, padding=1, bias=False)
    self.conv9 = nn.Conv2d(16, 16, 3, padding=1, bias=False)
    self.conv10 = nn.Conv2d(16, 16, 3, padding=1, bias=False)
    self.conv11 = nn.Conv2d(16, 16, 3, padding=1, bias=False)
    self.avpool = nn.AvgPool2d(2,2)
    self.fc1 = nn.Linear(8*8*16,10)
  def forward(self, x):
    padding_zeros_4 = torch.zeros(batch_size,4,16,16).to(device)
    padding_zeros_8 = torch.zeros(batch_size,8,16,16).to(device)
    x = self.conv1(x) #4*16*16
    identity = x.clone()
    x = F.relu(x)
    x = F.relu(self.conv2(x)) 
    x = self.conv3(x)
    x = x + identity
    identity = x.clone()
    x = F.relu(x)
    x = F.relu(self.conv4(x)) #8*16*16
    x = self.conv5(x) 
    x = x + torch.cat([identity,padding_zeros_4],axis=1)
    identity = x.clone()
    x = F.relu(x)
    x = F.relu(self.conv6(x)) 
    x = self.conv7(x)
    x = x + identity
    identity = x.clone() 
    x = F.relu(x)
    x = F.relu(self.conv8(x)) #16
    x = self.conv9(x) 
    x = x + torch.cat([identity,padding_zeros_8],axis=1)
    identity = x.clone() 
    x = F.relu(x)
    x = F.relu(self.conv10(x))
    x = self.conv11(x)
    x = x+identity
    x = F.relu(x)
    x = self.avpool(x)
    x = x.view(-1,8*8*16)
    x = self.fc1(x)
    return x #10 class scores

In [ ]:
# load weights
model = ResNet12()
model.to(device)
# model.load_state_dict(torch.load(save_to + model_name + '.pt' )) # for final model
model.load_state_dict(torch.load(save_to + model_name)) #for epoch 5 or overfited

<All keys matched successfully>

In [ ]:
# #load input and output 
# import pickle
# input = torch.zeros(1,3072).to(device)
# y = torch.empty(0).to(device)
# with torch.no_grad():
#   for image, label in train_loader:
#     image = image.to(device)
#     label = label.to(device)
#     y = torch.cat([y,label],axis = 0)
#     input = torch.cat([input,image.view(-1,3072)],axis=0)
# input = input[1:]

# print(y.shape)
# print(input.shape)

# input = input.cpu().numpy()
# y = y.cpu().numpy()
# pickle.dump(input, open(save_to+"np_input", 'wb'))
# pickle.dump(y, open(save_to+"np_y", 'wb'))

# Extract feature maps for each layer

In [ ]:
#register hook for every layer
layer_names = ['conv1', 'conv2', 'conv3', 'conv4', 'conv5', 'conv6', 'conv7', 'conv8', 'conv9', 'conv10', 'conv11', 'avpool', 'fc1']

dict_X = {}
model.conv2.register_forward_hook(get_activation('conv2'))
model.conv4.register_forward_hook(get_activation('conv4'))
model.conv6.register_forward_hook(get_activation('conv6'))
model.conv8.register_forward_hook(get_activation('conv8'))
model.conv10.register_forward_hook(get_activation('conv10'))
model.avpool.register_forward_hook(get_activation('avpool'))
model.fc1.register_forward_hook(get_activation('fc1'))

with torch.no_grad():
  for input, _ in train_loader:
    input = input.to(device)
    output = model(input)

In [ ]:
for layer in dict_X.keys():
  print(layer+': ', dict_X[layer].shape)
#save output for each layer
m = nn.ReLU()

for even_layer in ['conv2', 'conv4', 'conv6', 'conv8', 'conv10']:
  pickle.dump(m(dict_X[even_layer]).cpu().numpy(), open(save_to+model_name+'_'+even_layer+'_relu','wb'))
  del dict_X[even_layer]

pickle.dump(dict_X['avpool'].cpu().numpy(), open(save_to+model_name+'_avpool','wb'))
del dict_X['avpool']

pickle.dump(dict_X['fc1'].cpu().numpy(), open(save_to+model_name+'_fc1','wb'))
del dict_X['fc1']

conv2:  torch.Size([50000, 1024])
conv4:  torch.Size([50000, 2048])
conv6:  torch.Size([50000, 2048])
conv8:  torch.Size([50000, 4096])
conv10:  torch.Size([50000, 4096])
avpool:  torch.Size([50000, 1024])
fc1:  torch.Size([50000, 10])


In [ ]:
model.conv1.register_forward_hook(get_activation('conv1'))
model.conv3.register_forward_hook(get_activation('conv3'))
model.conv5.register_forward_hook(get_activation('conv5'))
model.conv7.register_forward_hook(get_activation('conv7'))
model.conv9.register_forward_hook(get_activation('conv9'))
model.conv11.register_forward_hook(get_activation('conv11'))

with torch.no_grad():
  for input, _ in train_loader:
    input = input.to(device)
    output = model(input)

In [ ]:
for layer in ['conv2', 'conv4', 'conv6', 'conv8', 'conv10', 'avpool', 'fc1']:
  del dict_X[layer]

for layer in dict_X.keys():
  print(layer+': ', dict_X[layer].shape)
#save output for each layer
m = nn.ReLU()


pickle.dump(m(dict_X['conv1']).cpu().numpy(), open(save_to+model_name+'_conv1_relu','wb'))

conv3_plus_identity = dict_X['conv3'].clone()+dict_X['conv1'].clone()
pickle.dump(m(conv3_plus_identity).cpu().numpy(), open(save_to+model_name+'_conv3+identity_relu','wb'))
del dict_X['conv1']
del dict_X['conv3']

padding_zeros_4 = torch.zeros(50000,1024).to(device)
conv5_plus_identity = dict_X['conv5'].clone()+torch.cat([conv3_plus_identity,padding_zeros_4],axis=1)
pickle.dump(m(conv5_plus_identity).cpu().numpy(), open(save_to+model_name+'_conv5+identity_relu','wb'))
del dict_X['conv5']
del conv3_plus_identity
del padding_zeros_4

conv7_plus_identity = dict_X['conv7'].clone()+conv5_plus_identity
pickle.dump(m(conv7_plus_identity).cpu().numpy(), open(save_to+model_name+'_conv7+identity_relu','wb'))
del dict_X['conv7']
del conv5_plus_identity

padding_zeros_8 = torch.zeros(50000,2048).to(device)
conv9_plus_identity = dict_X['conv9'].clone()+torch.cat([conv7_plus_identity,padding_zeros_8],axis=1)
pickle.dump(m(conv9_plus_identity).cpu().numpy(), open(save_to+model_name+'_conv9+identity_relu','wb'))
del dict_X['conv9']
del conv7_plus_identity
del padding_zeros_8

conv11_plus_identity = dict_X['conv11'].clone()+conv9_plus_identity
pickle.dump(m(conv11_plus_identity).cpu().numpy(), open(save_to+model_name+'_conv11+identity_relu','wb'))
del dict_X['conv11']
del conv9_plus_identity
del conv11_plus_identity

conv1:  torch.Size([50000, 1024])
conv3:  torch.Size([50000, 1024])
conv5:  torch.Size([50000, 2048])
conv7:  torch.Size([50000, 2048])
conv9:  torch.Size([50000, 4096])
conv11:  torch.Size([50000, 4096])
